In [5]:
import sys,re,collections

from oauth2client import client
from googleapiclient import sample_tools

In [36]:
class PI:
    
    def __init__(self, **kwargs):
        self.name = kwargs.get('name', '')
        self.dept = kwargs.get('dept', [])
        self.proj = kwargs.get('proj', [])
    
    def __repr__(self):
        return 'PI: {} '.format(self.name) + ','.join(self.dept) + ' ' + ','.join(self.proj)
    
    def add_dept(s):
        self.dept.append(s)
        
    def add_proj(self, s):
        self.proj.append(s)

In [37]:
LuiSha = PI(name='Lui Sha',dept=['CS'],proj=['2016-08-100-02'])
RomitChoudhury = PI(name='Romit Roy Choudhury',dept=['ECE'],proj=['2017-06-100-01'])
SayanMitra = PI(name='Sayan Mitra',dept=['ECE'],proj=['2016-06-103-01'])
GirishKrishnan = PI(name='Girish Krishnan',dept=['ISE'],proj=['2017-03-106-01'])
SethHutchinson = PI(name='Seth Hutchinson', dept=['ECE'],proj=['2016-10-103-02'])
GraceGao = PI(name='Grace Gao', dept=['AE'], proj=['2016-11-103-01'])


PI_list = [LuiSha, RomitChoudhury, SayanMitra, GirishKrishnan, SethHutchinson]

proj_mapping = collections.defaultdict(PI)
proj_mapping = {'2016-08-100-02': LuiSha, 
                '2017-06-100-01': RomitChoudhury,
                '2016-06-103-01': SayanMitra,
                '2017-03-106-01': GirishKrishnan,
                '2016-10-103-02': SethHutchinson}

for pi in PI_list:
    print(pi)

PI: Lui Sha CS 2016-08-100-02
PI: Romit Roy Choudhury ECE 2017-06-100-01
PI: Sayan Mitra ECE 2016-06-103-01
PI: Girish Krishnan ISE 2017-03-106-01
PI:   


In [6]:
proj_mapping = collections.defaultdict(list)
proj_mapping = {'2016-08-100-02': ['Lui Sha',['CS']], 
                '2017-06-100-01': ['Romit Roy Choudhury',['ECE']],
                '2016-06-103-01': ['Sayan Mitra',['ECE']],
                '2017-03-106-01': ['Girish Krishnan',['ISE']],
                ''}

In [7]:
proj_mapping['2016-08-100-02']

['Lui Sha', ['CS']]